# Nettoyage du jeu de données "Open Facts Food"

In [1]:
# importation des librairies nécessaires

import numpy as np
import pandas as pd
import scipy.stats as stats
import dask.dataframe as dd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

In [2]:
# importation du jeu de données
filename = "openfoodfacts_data.csv"
df = dd.read_csv(filename, dtype='object', sep='\t')

In [ ]:
# Transformation du dataframe Dask en dataframe Pandas
df = df.compute()

## 1) Traitement des valeurs manquantes

Toutes les colonnes où on observe un pourcentage supérieur à 75% sont supprimées du jeu de données. 

In [ ]:
df = df.loc[:, df.isnull().mean() <.75]

In [ ]:
# Affichage du pourcentage de donneés manquantes
df.isnull().mean() * 100

In [ ]:
%matplotlib inline
msno.matrix(df)

In [ ]:
df.head()

In [ ]:
# modification des types du dataset en float quand cela est nécessaire
for col in df.columns:
    if df[col].dtype == object:
        df[col] = pd.to_numeric(df[col].str.replace(',', '.'), errors='ignore')

In [ ]:
col_cat = ["additives_n", "ingredients_from_palm_oil_n", "ingredients_that_may_be_from_palm_oil_n",
           "nutriscore_score", "nova_group"]

for col in col_cat:
    df[col] = df[col].astype("object")

In [ ]:
df["created_datetime"] = pd.to_datetime(df["created_datetime"], format="%Y-%m-%d %H:%M:%S.%f", utc=True)
df['created_date'] = pd.to_datetime(df['created_datetime'], format='%d:%m:%Y').dt.date
df['created_years'] = pd.to_datetime(df['created_datetime'], format='%d:%m:%Y').dt.year

df["last_modified_datetime"] = pd.to_datetime(df["last_modified_datetime"], format="%Y-%m-%d %H:%M:%S.%f", utc=True)
df['last_modified_date'] = pd.to_datetime(df['last_modified_datetime'], format='%d:%m:%Y').dt.date
df['last_modified_years'] = pd.to_datetime(df['last_modified_datetime'], format='%d:%m:%Y').dt.year

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.shape

Pour finir le traitement des valeurs manquantes, il faudra choisir entre plusieurs méthodes : 
- Enlever les valeurs manquantes (via la méthode dropna)
- Remplacer les valeurs manquantes avec la médiane
- Remplacer les valeurs manquantes avec la moyenne

Avant de procéder à cette étape, on va s'intéresser à deux autres points : le traitement des donneés dupliquées ainsi que l'étude des anomalies.

## 2) Traitement des données dupliquées

Il y a deux types de données dupliquées : 
- Celles qui correspondent à des colonnes dont les noms sont différents mais donnent la même information (les colonnes _en ou _tags par exemple)
- Celles que l'on peut enlever à l'aide des méthodes. duplicated() et .drop_duplicates()

Dans un premier temps, nous allons nous créer une liste avec les noms des colonnes à supprimer (avec le suffixe _en ou _tags). Ensuite, on cherchera à supprimer les données dupliquées à l'aide des méthodes spécifiques des dataframes Pandas. 

#### a) Suppression des colonnes redondantes

In [ ]:
drop_cols = ["created_datetime", "last_modified_datetime", "created_t", "last_modified_t", "brands_tags", "categories_tags", "categories_en", "countries_tags", "countries_en", "states_tags", "states_en",
             "main_category_en", "image_small_url", "image_ingredients_small_url", "image_nutrition_small_url", "nutrition-score-fr_100g"]

In [ ]:
df.drop(drop_cols, axis=1, inplace=True)

#### b) Suppression des informations dupliquées avec .drop_duplicates() et .duplicated()

In [ ]:
dupl = df.duplicated()

In [ ]:
df_dupl = df[dupl]
df_dupl.head()

In [ ]:
df = df[~dupl]

In [ ]:
df.shape

Le traitement des données dupliquées montre qu'il y a trois variables qui étaient dupliquées.

## Détection des outliers

In [ ]:
for var in df.columns:
    if df[var].dtypes == 'float64':
        sns.boxplot(x=var, data=df)
        plt.title(var)
        plt.show()

## Traitement des outliers à l'aide de la médiane (IQR)

In [ ]:
q1 = df.quantile(0.25)
q3 = df.quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
print(lower_bound)
print(upper_bound)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

df_clean = df[~((df < lower_bound) | (df > upper_bound)).any(axis=1)]
df_clean.shape

In [ ]:
for var in df_clean.columns:
    if df_clean[var].dtypes == 'float64':
        sns.boxplot(x=var, data=df_clean)
        plt.title(var)
        plt.show()

In [ ]:
df_clean.describe()

## Traitement des valeurs manquantes (suite)

Nos valeurs ont été nettoyées des données aberrantes et on se rend compte que l'index glycémique ne contient qu'une seule valeur. On va donc enlever cette variable de notre étude. Il existe deux variables qui rassemblent les données du nutriscore et elles sont identiques. On va donc en supprimer une pour éviter la redondance dans notre dataset. 

### Retrait des valeurs manquantes

In [ ]:
df = df_clean.copy()

In [ ]:
df_clean_no_na = df.dropna()

In [ ]:
%matplotlib inline
msno.matrix(df_clean_no_na)

In [ ]:
df_clean_no_na.describe()

In [ ]:
df_clean_no_na.shape

In [ ]:
df_clean_no_na.to_csv("openfoodfacts_clean.csv", index=False)

### Remplacement des valeurs manquantes par la valeur médiane

In [ ]:
df_clean_med = df.copy()

In [ ]:
df_clean_med.fillna(df.median(), inplace=True)

In [ ]:
msno.matrix(df_clean_med)

In [ ]:
df_clean_med.describe()

In [ ]:
df_clean_med.shape

In [ ]:
df_clean_med.to_csv("openfoodfacts_clean_med.csv", index=False)

### Remplacement des valeurs manquantes par la valeur moyenne

In [ ]:
df_clean_mean = df.copy()

In [ ]:
df_clean_mean.fillna(df.mean(), inplace=True)

In [ ]:
msno.matrix(df_clean_mean)

In [ ]:
df_clean_mean.describe()

In [ ]:
df_clean_mean.shape

In [ ]:
df_clean_mean.to_csv("openfoodfacts_clean_mean.csv", index=False)